## Морфология
#### План семинара:

1. Mystem консольный
2. Mystem через python
3. Pymorphy
4. NLTK


Нужные пакеты для этого семинара:

``pip install pymystem3``

``pip install pymorphy2``

``pip install nltk`` (Возможны проблемы для Windows)

# Mystem

Mystem - это свободно распространяемый морфологический анализатор для русского языка с закрытым исходным кодом.

My-stem значит my stemmer, стемминг -- это разбиение формы на основу и флексию. На самом деле Mystem может гораздо больше: устанавливать словарную форму слова, определять часть речи и грамматическую форму слова. В последних версиях Mystem умеет и выбирать из нескольких возможных грамматических разборов один, наиболее верный.

У Mystem нет графического оконного интерфейса

## Mystem консольный

**Preface:**

Программа для морфологического анализа **Mystem** запускается через командную строку и требует передачи определённых *аргументов* (или ещё их называют *параметрами*). Эти аргументы пишутся после пути к программе через пробел. Если сам аргумент тоже содержит пробел, его нужно обернуть в кавычки: `C:\some_programm.exe argument1 "argument 2"`

Среди аргументов есть такие, которые принято называть *опциями* или *ключами*, они начинаются с символа дефиса: `C:\some_prog.exe -a -b`. Их можно "склеивать": `C:\some_prog.exe -ab`.

**Как скачать**

Скачать Mystem можно [отсюда](https://tech.yandex.ru/mystem/), а [тут](https://tech.yandex.ru/mystem/doc/index-docpage/) лежит его документация (там описаны различные возможности вызова программы).

Примеры, которые приведены в документации, рассчитаны на пользователя Unix-подобной операционной системы. В начале примеров вызова стоит знак доллара, $. Доллар -- это аналог приглашения командной строки, просто не в Windows, а в Unix-подобных системах. Если вы берете примеры вызова со страницы документации за основу, игнорируйте знак доллара.

В документации написано: `$ mystem input`. На практике для пользователей Windows это будет значить что-то вроде `C:\mystem.exe input.txt`. 

В документации написано "стандартный ввод" и "стандартный вывод", это значит то, что вводится в командной строке или выводится в тот же терминал. Если не используются стандартный ввод и вывод, то используются файлы (выводной файл Mystem способен создать сам).

В 3-й версии Mystem кодировка по умолчанию -- utf-8. В первых версиях -- cp1251. Кодировка по умолчанию в командной строке Windows -- cp866. Из-за этого Mystem может не понимать слова, которые попадают к нему из стандартного ввода.

**Как запустить**

Особое внимание нужно уделить опции `-d`, она заставляет анализатор выбирать только один разбор из возможных. При этом выбор происходит только между разными частями речи. Если у одной части речи возможны разные разборы (например, разные падежи одного и того же существительного), то эти разборы не отбрасываются. Иначе говоря, Mystem снимает только частеречную омонимию. Омонимию форм он не снимает.

**Пример запуска из командной строки**

Возьмите файл experiment.txt и запустите его так, чтобы для каждого слова была снята омонимия и напечатана грамматическая информация.

Посмотрим, как Mystem справится с глокой куздрой и бокрёнком.

## Mystem через python

Для запуска сторонних программ, имеющих интерфейс командной строки, в питоне есть system, предоставляемая модулем os: `os.system("C:\mystem.exe input.txt output.txt")`. Но не забудьте импортировать модуль os.

Этот код берёт из директории *input_texts* все лежащие в ней файлы, отдаёт на разметку майстему и кладёт результат в соседнюю директорию *output_texts*.

In [4]:
import os

inp = "./input_files"
outp = "./output_files"
mystem_path = os.path.join('/Users/dkbrz/Downloads', 'mystem')
files = os.listdir(inp)
for fname in files:
    # os.path.abspath - находит абсолютный путь
    # os.path.join - объединяет части пути до файла
    input_filename = os.path.join(os.path.abspath(inp), fname)
    output_filename = os.path.join(os.path.abspath(outp), fname)
    os.system(f"{mystem_path} {input_filename} {output_filename}")

А что там на самом деле пишется? Посмотрим на последних переменных

In [5]:
f"{mystem_path} {input_filename} {output_filename}"

'/Users/dkbrz/Downloads/mystem /Users/dkbrz/hse-python/seminars/morphology/input_files/nabokov_4.txt /Users/dkbrz/hse-python/seminars/morphology/output_files/nabokov_4.txt'

Кстати, этот способ записи называется f-string и позволяет собирать строчки по шаблону

Это тоже можно попробовать. Для примера у нас в папке input_texts лежат нарезанные тексты Набокова Дар

## Библиотека pymystem3

Можно запускать mystem и с помощью специального модуля, **pymystem3**. Это проще и удобнее, потому что с тем, что выдаёт mystem, можно сразу работать как с питоновскими структурами данных. Но медленнее. Иногда гораздо-гораздо медленнее, чем разметить один файл mystem'ом сразу.   

**НО!**  
[В этой небольшой статье рассказывается как можно это обойти и в чем причина такой задержки.](https://habr.com/ru/post/503420/)

In [5]:
from pymystem3 import Mystem

In [7]:
mystem = Mystem()

У этого класса Mystem() есть два метода:

* lemmatize, возвращающий список лемм,
* и analyze, возвращающий полные разборы в виде словаря.

Возьмем небольшой текст и опробуем на нем эти два метода:

In [9]:
text = "Но не становится ли событие тем значительнее и исключительнее," +\
"чем большее число случайностей приводит к нему?" +\
" Лишь случайность может предстать перед нами как послание." +\
" Все, что происходит по необходимости, что ожидаемо, что повторяется всякий день, то немо." +\
" Лишь случайность о чем-то говорит нам. Мы стремимся прочесть ее, " +\
"как читают цыганки по узорам, начертанным кофейной гущей на дне чашки."

In [10]:
lemmas = mystem.lemmatize(text)
lemmas[10:20]

['тем', ' ', 'значительный', ' ', 'и', ' ', 'исключительный', ',', 'чем', ' ']

Можно собрать лемматизированный текст обратно:

In [11]:
print(''.join(lemmas))

но не становиться ли событие тем значительный и исключительный,чем больший число случайность приводить к он? лишь случайность мочь представать перед мы как послание. все, что происходить по необходимость, что ожидать, что повторяться всякий день, то немо. лишь случайность о что-то говорить мы. мы стремиться прочитывать она, как читать цыганка по узор, начертать кофейный гуща на дно чашка.



In [12]:
from pprint import pprint

In [13]:
ana = mystem.analyze(text)
pprint(ana[:20])

[{'analysis': [{'gr': 'CONJ=', 'lex': 'но', 'wt': 0.9998906299}], 'text': 'Но'},
 {'text': ' '},
 {'analysis': [{'gr': 'PART=', 'lex': 'не', 'wt': 1}], 'text': 'не'},
 {'text': ' '},
 {'analysis': [{'gr': 'V,нп=непрош,ед,изъяв,3-л,несов',
                'lex': 'становиться',
                'wt': 1}],
  'text': 'становится'},
 {'text': ' '},
 {'analysis': [{'gr': 'PART=', 'lex': 'ли', 'wt': 0.7719288688}], 'text': 'ли'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,сред,неод=(вин,ед|им,ед)', 'lex': 'событие', 'wt': 1}],
  'text': 'событие'},
 {'text': ' '},
 {'analysis': [{'gr': 'CONJ=', 'lex': 'тем', 'wt': 0.0857739759}],
  'text': 'тем'},
 {'text': ' '},
 {'analysis': [{'gr': 'A=срав', 'lex': 'значительный', 'wt': 0.2062520859}],
  'text': 'значительнее'},
 {'text': ' '},
 {'analysis': [{'gr': 'CONJ=', 'lex': 'и', 'wt': 0.9999770357}], 'text': 'и'},
 {'text': ' '},
 {'analysis': [{'gr': 'A=срав', 'lex': 'исключительный', 'wt': 1}],
  'text': 'исключительнее'},
 {'text': ','},
 {'analysi

Разбор для каждого слова является элементом массива:

In [34]:
word_example = ana[12]
print(type(word_example))
pprint(word_example)

<class 'dict'>
{'analysis': [{'gr': 'A=срав', 'lex': 'значительный', 'wt': 0.2062520859}],
 'text': 'значительнее'}


In [35]:
# Не всегда словарь имеет ключ 'analysis'. 
# Для пунктуации, пробелов и если pymystem не удается проанализровать слово, то такого ключа может и не быть.
word_example.keys() 

dict_keys(['analysis', 'text'])

In [36]:
word_example['text']

'значительнее'

In [37]:
word_example['analysis']

[{'lex': 'значительный', 'wt': 0.2062520859, 'gr': 'A=срав'}]

In [41]:
tags = word_example['analysis'][0]['gr']
print(f'{type(tags) = }')
print(tags)

type(tags) = <class 'str'>
A=срав


В ключе **`text`** можно найти исходное слово, в **`analysis`** - грамматические характеристики и леммы, но его может и не быть.
Для пунктуации, пробелов и если pymystem не удается проанализировать слово, то ключ **`analysis`** отсутствует в ответе.   
В грамматическом разборе знаком ``=`` отделяются изменяемые характеристики от неизменяемых. Знаком ``|`` отделяются омонимичные разборы.  
[**Ссылка на расшифровку граммем.**](https://yandex.ru/dev/mystem/doc/grammemes-values.html)

In [16]:
# Достанем все части речи:
for word in ana:
    if 'analysis' in word:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        print(word['text'], pos)

Но CONJ
не PART
становится V
ли PART
событие S
тем CONJ
значительнее A
и CONJ
исключительнее A
чем CONJ
большее A
число S
случайностей S
приводит V
к PR
нему SPRO
Лишь PART
случайность S
может V
предстать V
перед PR
нами SPRO
как CONJ
послание S
Все SPRO
что CONJ
происходит V
по PR
необходимости S
что CONJ
ожидаемо V
что CONJ
повторяется V
всякий APRO
день S
то CONJ
немо ADV
Лишь PART
случайность S
о PR
чем-то SPRO
говорит V
нам SPRO
Мы SPRO
стремимся V
прочесть V
ее SPRO
как ADVPRO
читают V
цыганки S
по PR
узорам S
начертанным V
кофейной A
гущей S
на PR
дне S
чашки S


## Итог:

**Достоинства Mystem'a:**

- хорошее качество разбора
- по умолчанию разрешается частеречная омонимия (внутри части речи остается)
- при разборе учитывается контекст
- совместим с разметкой НКРЯ

**Недостатки Mystem'a:**

- медленный, но есть способы ускорить

# Pymorphy

Может делать то же, что и pymystem3, и даже больше: изменять слова в нужную форму (спрягать и склонять). При этом pymorphy2 справляется и с незнакомыми словами.

[Документация](https://pymorphy2.readthedocs.io/en/latest/)

pymorphy2 обучался на словарях проекта OpenCorpora, что накладывается определённый отпечаток на его тагсет (например, он не очень хорошо совместим с тагсетом Mystem).

Для работы точно так же надо создать экземпляр класса ``MorphAnalyzer``. Рекомендуется создать один экземпляр и дальше с ним и работать, поскольку он занимает достаточно много памяти, и если создать несколько экземпляров анализаторов, то они будут тормозить программу.

In [45]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Разбор слова делается при помощи метода parse:

In [56]:
morph.parse('мама мыла раму')

[Parse(word='мама мыла раму', tag=OpencorporaTag('NOUN,anim,masc,Name sing,datv'), normal_form='мама мыла рам', score=1.0, methods_stack=((FakeDictionary(), 'мама мыла раму', 27, 2), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'раму')))]

In [46]:
ana = morph.parse('стекло')
ana

[Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,nomn'), normal_form='стекло', score=0.690476, methods_stack=((DictionaryAnalyzer(), 'стекло', 157, 0),)),
 Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,accs'), normal_form='стекло', score=0.285714, methods_stack=((DictionaryAnalyzer(), 'стекло', 157, 3),)),
 Parse(word='стекло', tag=OpencorporaTag('VERB,perf,intr neut,sing,past,indc'), normal_form='стечь', score=0.023809, methods_stack=((DictionaryAnalyzer(), 'стекло', 1015, 3),))]

Как видно, анализатор вернул все возможные разборы этого слова, отранжировав их по вероятности.

У каждого разбора есть атрибуты: исходное слово, тэг, лемма, вероятность разбора:

In [47]:
first = ana[0]  # первый разбор
print('Слово:', first.word)
print('Тэг:', first.tag)
print('Лемма:', first.normal_form)
print('Вероятность:', first.score)

Слово: стекло
Тэг: NOUN,inan,neut sing,nomn
Лемма: стекло
Вероятность: 0.690476


Для каждого разбора можно получить лемму и всю информацию о ней (т.е. еще один разбор, только уже для леммы):

In [48]:
first.normalized

Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,nomn'), normal_form='стекло', score=0.690476, methods_stack=((DictionaryAnalyzer(), 'стекло', 157, 0),))

In [49]:
last = ana[-1] # последний разбор
print('Разбор слова: ', last)
print()
print('Разбор леммы: ', last.normalized)

Разбор слова:  Parse(word='стекло', tag=OpencorporaTag('VERB,perf,intr neut,sing,past,indc'), normal_form='стечь', score=0.023809, methods_stack=((DictionaryAnalyzer(), 'стекло', 1015, 3),))

Разбор леммы:  Parse(word='стечь', tag=OpencorporaTag('INFN,perf,intr'), normal_form='стечь', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стечь', 1015, 0),))


Если распечатать тег разбора, то может показаться, что это строка:

In [50]:
first = ana[0]  # первый разбор
print(first.tag)

NOUN,inan,neut sing,nomn


Но на самом деле это объект класса OpencorporaTag, так что некоторые вещи, которые можно делать со строками, с тэгами делать нельзя. А некоторые все-таки можно.

Например, можно проверить, есть ли какая-то граммема в теге:

In [51]:
'NOUN' in first.tag

True

In [52]:
'VERB' in first.tag

False

In [53]:
{'NOUN', 'inan'} in first.tag

True

Из каждого тега можно достать более дробную информацию. Если граммема есть в разборе, то вернется ее значение, если ее нет, то вернется None.

In [54]:
print(last.tag)
print('Время: ', last.tag.tense)
print('Падеж: ', last.tag.case)

VERB,perf,intr neut,sing,past,indc
Время:  past
Падеж:  None


Список граммем, которые используются в модуле, находится здесь - https://pymorphy2.readthedocs.io/en/latest/user/grammemes.html.

Если искать какую-то граммему, которой нет в этом списке, возникнет ошибка.

Можно получить строку с кириллическими обозначениями граммем:

In [55]:
first.tag.cyr_repr

'СУЩ,неод,ср ед,им'

## Словоизменение

Если у нас есть разбор слова, то мы можем это слово поставить в другую форму с помощью функции inflect. Эта функция получает на вход множество граммем и пытается применить их к нашему разбору.

In [58]:
morph.parse('программирую')

[Parse(word='программирую', tag=OpencorporaTag('VERB,impf,tran sing,1per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'программирую', 171, 1),))]

In [59]:
prog = morph.parse('программирую')[0]
prog.inflect({'plur'})

Parse(word='программируем', tag=OpencorporaTag('VERB,impf,tran plur,1per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'программируем', 171, 2),))

In [60]:
prog.inflect({'plur', 'past'})

Parse(word='программировали', tag=OpencorporaTag('VERB,impf,tran plur,past,indc'), normal_form='программировать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'программировали', 171, 10),))

In [61]:
prog.inflect({'past'})

Parse(word='программировал', tag=OpencorporaTag('VERB,impf,tran masc,sing,past,indc'), normal_form='программировать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'программировал', 171, 7),))

In [62]:
prog.inflect({'past', 'femn'})[0]

'программировала'

**Формы слова**

С помощью атрибута lexeme можно получить массив всех форм слова:

In [63]:
prog.lexeme[:5]

[Parse(word='программировать', tag=OpencorporaTag('INFN,impf,tran'), normal_form='программировать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'программировать', 171, 0),)),
 Parse(word='программирую', tag=OpencorporaTag('VERB,impf,tran sing,1per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'программирую', 171, 1),)),
 Parse(word='программируем', tag=OpencorporaTag('VERB,impf,tran plur,1per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'программируем', 171, 2),)),
 Parse(word='программируешь', tag=OpencorporaTag('VERB,impf,tran sing,2per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'программируешь', 171, 3),)),
 Parse(word='программируете', tag=OpencorporaTag('VERB,impf,tran plur,2per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'программируете', 171, 4),))]

**Согласование слов с числительными**

Из документации:

    Слово нужно ставить в разные формы в зависимости от числительного, к которому оно относится. Например: “1 бутявка”, “2 бутявки”, “5 бутявок” Для этих целей используйте метод Parse.make_agree_with_number():

In [64]:
butyavka = morph.parse('бутявка')[0]

In [65]:
butyavka.make_agree_with_number(1).word

'бутявка'

In [66]:
butyavka.make_agree_with_number(2).word

'бутявки'

In [67]:
butyavka.make_agree_with_number(5).word

'бутявок'

## Итог

**Достоинства Pymorphy:**

- умеет составлять разборы, находить лемму, склонять и спрягать
- генерирует гипотезы для незнакомых слов
- может работать с украинским языком (но словари нужно отдельно устанавливать)

**Недостатки Pymorphy:**

- качество хуже, чем у Mystem
- работает только на уровне отдельных слов (и естественно, не учитывает контекст)

**Небольшой хак**

Pymorphy и так работает очень быстро, но можно еще быстрее, если мы будем сохранять разборы для очень популярных слов

# Сравнение скорости лемматизации

In [90]:
from string import punctuation

In [69]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [91]:
text = open('./input_files/nabokov.txt').read()
text = [word.lower().strip(punctuation) for word in text.split()]
text = [word for word in text if word != '']

### Чистый Pymorphy

In [92]:
%%timeit
lemmas = []

for word in text:
    lemmas.append(morph.parse(word)[0].normal_form)

4.62 s ± 124 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Pymorphy + запоминание слов

In [79]:
%%timeit
lemmas = []
known_words = {}

for word in text:
    if word in known_words:
        lemmas.append(known_words[word])
    else:
        result = morph.parse(word)[0].normal_form
        lemmas.append(result)
        known_words[word] = result

1.26 s ± 76.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Мы просто запоминаем леммы и поэтому не парсим слово каждый раз, а берем из быстрого хранилища готовый результат. Это может серьезно загружать память (при больших объемах), но значительно сократит время работы.

### Pymorphy + кэширование запусков
Вариант ниже похож на тот где мы использовали словарь, в котором хранили те слова, которые уже встречались ранее.  
Толь здесь это сделанно на уровне кэшировования. То есть мы храними историю запусков нужной нам функции при заданных параметрах.  
Если в кэше мы находим что данная функция уже запускалась с такими параметрами, то берем результат из истории запусков.  
Если комбинация параметров новая, то запускаем исходную функцию.

LRU - Least Recently Used, удаляет из сохраненного справочника запусков наиболее редкие запуски.

In [83]:
from functools import lru_cache

In [85]:
%%timeit
lemmas = []

@lru_cache(maxsize=1000000)
def parse_word(word):
    morph.parse(word)[0].normal_form
        
for word in text:
    lemmas.append(parse_word(word))

1.29 s ± 41.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Pymystem

In [102]:
text = open('./input_files/nabokov.txt').read()

In [103]:
%%timeit
lemmas = mystem.lemmatize(text)

805 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**НО!** Если мы будем запускать лемматизацию по словам, то скорость снизится + мы потеряем тот бонус, что pymystem учитывает контекст.
Поэтому иногда выгодно придумать какой то уникальный разделить между отдельными текстами и объединить их используя этот разделитель.  
А объединенный текст подать на вход в mystem.

In [104]:
text = open('./input_files/nabokov.txt').read()
text = [word.lower().strip(punctuation) for word in text.split()]
text = [word for word in text if word != '']

In [105]:
%%timeit
lemmas = []

for word in text:
    lemmas.append(mystem.lemmatize(word))

1.37 s ± 93.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### NLTK
Это уже не просто морфологический анализатор, а целая NLP библиотека!  
Что мы тут можем делать? Можем тоекнизировать какой-нибудь текст.

In [106]:
text = """
Такой зеленый, серый, то есть
весь заштрихованный дождем,
и липовое, столь густое,
что я перенести - уйдем!
Уйдем и этот сад оставим
и дождь, кипящий на тропах
между тяжелыми цветами,
целующими липкий прах.
Уйдем, уйдем, пока не поздно,
скорее, под плащом, домой,
пока еще ты не опознан,
безумный мой, безумный мой!

Держусь, молчу. Но с годом каждым,
под гомон птиц и шум ветвей,
разлука та обидней кажется,
обида кажется глупей.
И все страшней, что опрометчиво
проговорюсь и перебью
теченье тихой, трудной речи,
давно проникшей в жизнь мою.

Над краснощекими рабами
лазурь как лаковая вся,
с накачанными облаками,
едва заметными толчками
передвигающимися.
Ужель нельзя там притулиться
и нет там темного угла,
где темнота могла бы слиться
с иероглифами крыла?
Так бабочка не шевелится
пластом на плесени ствола.

Какой закат! И завтра снова,
и долго-долго быть жаре,
что безошибочно основано
на тишине и мошкаре.
В луче вечернем повисая,
она толчется без конца,
как бы игрушка золотая
в руках немого продавца.

Как я люблю тебя. Есть в этом
вечернем воздухе порой
лазейки для души, просветы
в тончайшей ткани мировой.
Лучи проходят меж стволами.
Как я люблю тебя! Лучи
проходят меж стволами, пламенем
ложатся на стволы. Молчи.
Замри под веткою расцветшей,
вдохни, какое разлилось -
зажмурься, уменьшись и в вечное
пройди украдкою насквозь.
"""

In [107]:
import nltk
from nltk.tokenize import word_tokenize

%time print(word_tokenize(text))

['Такой', 'зеленый', ',', 'серый', ',', 'то', 'есть', 'весь', 'заштрихованный', 'дождем', ',', 'и', 'липовое', ',', 'столь', 'густое', ',', 'что', 'я', 'перенести', '-', 'уйдем', '!', 'Уйдем', 'и', 'этот', 'сад', 'оставим', 'и', 'дождь', ',', 'кипящий', 'на', 'тропах', 'между', 'тяжелыми', 'цветами', ',', 'целующими', 'липкий', 'прах', '.', 'Уйдем', ',', 'уйдем', ',', 'пока', 'не', 'поздно', ',', 'скорее', ',', 'под', 'плащом', ',', 'домой', ',', 'пока', 'еще', 'ты', 'не', 'опознан', ',', 'безумный', 'мой', ',', 'безумный', 'мой', '!', 'Держусь', ',', 'молчу', '.', 'Но', 'с', 'годом', 'каждым', ',', 'под', 'гомон', 'птиц', 'и', 'шум', 'ветвей', ',', 'разлука', 'та', 'обидней', 'кажется', ',', 'обида', 'кажется', 'глупей', '.', 'И', 'все', 'страшней', ',', 'что', 'опрометчиво', 'проговорюсь', 'и', 'перебью', 'теченье', 'тихой', ',', 'трудной', 'речи', ',', 'давно', 'проникшей', 'в', 'жизнь', 'мою', '.', 'Над', 'краснощекими', 'рабами', 'лазурь', 'как', 'лаковая', 'вся', ',', 'с', 'накач

Можем разделить текст на предложения (сплиттинг):

In [108]:
from nltk.tokenize import sent_tokenize

sent_tokenize(text)

['\nТакой зеленый, серый, то есть\nвесь заштрихованный дождем,\nи липовое, столь густое,\nчто я перенести - уйдем!',
 'Уйдем и этот сад оставим\nи дождь, кипящий на тропах\nмежду тяжелыми цветами,\nцелующими липкий прах.',
 'Уйдем, уйдем, пока не поздно,\nскорее, под плащом, домой,\nпока еще ты не опознан,\nбезумный мой, безумный мой!',
 'Держусь, молчу.',
 'Но с годом каждым,\nпод гомон птиц и шум ветвей,\nразлука та обидней кажется,\nобида кажется глупей.',
 'И все страшней, что опрометчиво\nпроговорюсь и перебью\nтеченье тихой, трудной речи,\nдавно проникшей в жизнь мою.',
 'Над краснощекими рабами\nлазурь как лаковая вся,\nс накачанными облаками,\nедва заметными толчками\nпередвигающимися.',
 'Ужель нельзя там притулиться\nи нет там темного угла,\nгде темнота могла бы слиться\nс иероглифами крыла?',
 'Так бабочка не шевелится\nпластом на плесени ствола.',
 'Какой закат!',
 'И завтра снова,\nи долго-долго быть жаре,\nчто безошибочно основано\nна тишине и мошкаре.',
 'В луче вечернем

NLTK может удалять стоп слова. Стоп-слова - это высокочастотные союзы, предлоги и другие служебные части речи, которые не дают нам никакой информации о конкретном тексте. В NLTK есть готовые списки стоп-слов (да-да, и для русского тоже есть).

In [ ]:
nltk.download('stopwords')

In [111]:
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/u14510182/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [112]:
from nltk.corpus import stopwords

# загружаем нужный список стоп-слов
sw = stopwords.words('russian')

# смотрим, что внутри
print(sw)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [113]:
# токенизируем текст, приводим к нижнему регистру и оставляем только последовательности из букв,
# т.е. все токены, где были знаки препинания и числа, исчезнут
words = [w.lower() for w in word_tokenize(text) if w.isalpha()]

# какие слова исчезли?
filtered = [w for w in words if w not in sw]
print(filtered)

['зеленый', 'серый', 'весь', 'заштрихованный', 'дождем', 'липовое', 'столь', 'густое', 'перенести', 'уйдем', 'уйдем', 'сад', 'оставим', 'дождь', 'кипящий', 'тропах', 'тяжелыми', 'цветами', 'целующими', 'липкий', 'прах', 'уйдем', 'уйдем', 'пока', 'поздно', 'скорее', 'плащом', 'домой', 'пока', 'опознан', 'безумный', 'безумный', 'держусь', 'молчу', 'годом', 'каждым', 'гомон', 'птиц', 'шум', 'ветвей', 'разлука', 'та', 'обидней', 'кажется', 'обида', 'кажется', 'глупей', 'страшней', 'опрометчиво', 'проговорюсь', 'перебью', 'теченье', 'тихой', 'трудной', 'речи', 'давно', 'проникшей', 'жизнь', 'мою', 'краснощекими', 'рабами', 'лазурь', 'лаковая', 'вся', 'накачанными', 'облаками', 'едва', 'заметными', 'толчками', 'передвигающимися', 'ужель', 'притулиться', 'темного', 'угла', 'темнота', 'могла', 'слиться', 'иероглифами', 'крыла', 'бабочка', 'шевелится', 'пластом', 'плесени', 'ствола', 'закат', 'завтра', 'снова', 'жаре', 'безошибочно', 'основано', 'тишине', 'мошкаре', 'луче', 'вечернем', 'повисая

И наконец-то стемминг

In [114]:
# умеет работать не только с английским текстом

from nltk.stem.snowball import SnowballStemmer

snowball = SnowballStemmer("russian")

In [115]:
ruswords = set(word_tokenize(text))

for w in sorted(ruswords)[20:50]:
    print("%s: %s" % (w, snowball.stem(w)))

бабочка: бабочк
без: без
безошибочно: безошибочн
безумный: безумн
бы: бы
быть: быт
в: в
вдохни: вдохн
весь: ве
ветвей: ветв
веткою: ветк
вечернем: вечерн
вечное: вечн
воздухе: воздух
все: все
вся: вся
где: где
глупей: глуп
годом: год
гомон: гомон
густое: густ
давно: давн
для: для
дождем: дожд
дождь: дожд
долго-долго: долго-долг
домой: дом
души: душ
едва: едв
есть: ест


По-моему, качество -- просто дно.

А в лемматизации тут нет русского, но в целом good to know.

In [118]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/u14510182/nltk_data...


True

In [119]:
from nltk import WordNetLemmatizer

wnl = WordNetLemmatizer()

In [120]:
wnl.lemmatize('running', pos='v')

'run'

## Задание

Текст - первая глава произведения "Дар" Набокова. Файл nabokov.txt у нас в репозитории.

1. Первое задание - это небольшой эксперимент. Возьмите пять любых абзацев из текста и распарсите их двумя способами. 1) просто через pymystem (.analyze) и 2) через pymystem, предварительно почистив текст от пунктуации (тоже .analyze). Замерить, что из этого быстрее с помощью line magic ``%time some_python_expression_here``
2. Токенизируйте весь текст с помощью nltk. 
3. Почистите его от знаков препинания (тут пригодится список из первого задания), стоп-слов (с помощью nltk) и слов не на кириллице. Сделайте регистр lower у всх слов.
4. Лемматизируйте с помощью pymorphy (.normal_form)
5. Cоставьте частотный список слов. Выведите 20 самых частотных слов вообще.
6. Найдите 20 самых частотных существительных.
7. В тексте (списку слов), полчившемся после пункта 3 (токенизированному и почищенному), поищите биграммы. Для этого нужно будет посмотреть nltk документацию про nltk.bigrams(). Выведите 10 самых частотных биграммов.
 
Напоминание:

**N-граммы** — это сочетания из N элементов (слов, символов), идущих друг за другом. Одиночные элементы называются униграммами, сочетания из двух элементов — биграммами, из трёх — триграммами, а дальше все пишется цифрами: 4-граммы, 5-граммы и т.д.